In [1]:
### ALL NECESSARY LIBRARIES ###
import pandas as pd
import warnings
import json
warnings.simplefilter(action='ignore')

# for saving variables
import pickle
import os
import tiktoken

# needed for gpt
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt

%run "./utilityFunctions.ipynb"

In [46]:
# import data
transcripts = pd.read_csv('ellieAndParticipantTranscripts.csv')

with open('e_daic_phq8_scores_lang.pkl', 'rb') as f:
    phq8_scores = pickle.load(f)


# rename column Participant_ID to ParticipantID
phq8_scores.rename(columns={'Participant_ID': 'ParticipantID'}, inplace=True)

merged_data = pd.merge(transcripts, phq8_scores, on='ParticipantID')
merged_data.head()
merged_data.to_csv('PHQ8_Transcript_Combined.csv', index=False)
phq_binary_distribution = merged_data['PHQ_Binary'].value_counts()
phq_binary_distribution 

merged_data


,ParticipantID,Conversation,Gender,PHQ_Binary,PHQ_Score,PCL-C (PTSD),PTSD Severity,PHQ_8NoInterest,PHQ_8Depressed,PHQ_8Sleep,PHQ_8Tired,PHQ_8Appetite,PHQ_8Failure,PHQ_8Concentrating,PHQ_8Moving,PHQ_8Total,PHQ8_Binary,PHQ8_Score
0,300,Ellie: how are you doing today. Participant: g...,male,0,2,0,25.0,"Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...","Several days over the last 2 weeks, I was both...","Over the last 2 weeks, I was not at all bother...","Several days over the last 2 weeks, I was both...","Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...",2,This patient has not been diagnosed with depre...,Total PHQ8 score is 2.
1,301,Ellie: how are you doing today. Participant: i...,male,0,3,0,17.0,"Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...","Several days over the last 2 weeks, I was both...","Several days over the last 2 weeks, I was both...","Several days over the last 2 weeks, I was both...","Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...",3,This patient has not been diagnosed with depre...,Total PHQ8 score is 3.
2,302,Ellie: how are you doing today. Participant: i...,male,0,4,0,28.0,"Several days over the last 2 weeks, I was both...","Over the last 2 weeks, I was not at all bother...","Nearly every day over the last 2 weeks, I was ...","Several days over the last 2 weeks, I was both...","More than half the days over the last 2 weeks,...","Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...",4,This patient has not been diagnosed with depre...,Total PHQ8 score is 4.
3,303,Ellie: how are you doing today. Participant: o...,female,0,0,0,17.0,"Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...","Several days over the last 2 weeks, I was both...","Several days over the last 2 weeks, I was both...","More than half the days over the last 2 weeks,...","More than half the days over the last 2 weeks,...","Several days over the last 2 weeks, I was both...","Over the last 2 weeks, I was not at all bother...",0,This patient has not been diagnosed with depre...,Total PHQ8 score is 0.
4,304,Ellie: how are you doing today. Participant: i...,female,0,6,0,20.0,"Several days over the last 2 weeks, I was both...","Nearly every day over the last 2 weeks, I was ...","More than half the days over the last 2 weeks,...","Nearly every day over the last 2 weeks, I was ...","Several days over the last 2 weeks, I was both...","More than half the days over the last 2 weeks,...","Over the last 2 weeks, I was not at all bother...","Several days over the last 2 weeks, I was both...",6,This patient has not been diagnosed with depre...,Total PHQ8 score is 6.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,488,Ellie: so how are you doing today. Participant...,female,0,0,0,20.0,"Several days over the last 2 weeks, I was both...","Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...","Several days over the last 2 weeks, I was both...","Over the last 2 weeks, I was not at all bother...","Several days over the last 2 weeks, I was both...","Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...",0,This patient has not been diagnosed with depre...,Total PHQ8 score is 0.
182,489,Ellie: so how are you doing today. Participant...,male,0,3,0,26.0,"Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I was not at all bother...","Over the last 2 weeks, I wa

In [49]:
conversation_history = [
    {"role": "system", "content": """
You are a professional psychologist who is very compassionate and empathetic
"""

    }
]


In [42]:
# Pricing details
PROMPT_COST_PER_1000_TOKENS = 0.005
COMPLETION_COST_PER_1000_TOKENS = 0.015

# Initialize the AzureOpenAI client with the specified endpoint and API key
client = AzureOpenAI(
    azure_endpoint="https://gptshuhaotest.openai.azure.com/",  # your Azure endpoint
    api_key="d0a5a4feefc34f71b685fc394d033b2c",  # your API key
    api_version="2024-02-01"
)

MODEL = "gpt-4o"  # change model here

# # Initialize conversation history with a system message
# conversation_history = [
#     {"role": "system", "content": "You are an expert on Animals"}
# ]

# keep track of conversation
def add_message(role, content):
    conversation_history.append({"role": role, "content": content})
    
# clear conversation  
def clear_conversation():
    global conversation_history
    conversation_history = [
        {"role": "system", "content": "You are a professional psychologist who is very compassionate and empathetic"}
    ]


# @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
def get_response(prompt):
    # Add the user's message to the conversation history.
    add_message("user", prompt)
    
    # Create a chat completion request to the OpenAI API.
    response = client.chat.completions.create(
        model=MODEL,     
        # The conversation history, including system, user, and assistant messages.
        messages=conversation_history, 
        temperature=1,  # Controls the randomness of the output. 
        max_tokens=1500,  # The maximum number of tokens to generate in the response.
        top_p=1,  # Controls the diversity via nucleus sampling; 1 means use all tokens.
        frequency_penalty=2,  # Penalizes new tokens based on their frequency in the text so far.
        presence_penalty=2,  # Penalizes new tokens based on whether they appear in the text so far.
        n=1,  # The number of completions to generate. Here, we are generating only one.
        response_format={ "type": "json_object" },
    )

    # Add the assistant's message to the conversation history.
    assistant_message = response.choices[0].message.content
    add_message("assistant", assistant_message)
    
    
    # Save usage and compute cost
    usage = response.usage
    computeCost(usage)

    # Return the assistant's message.
    return assistant_message




In [47]:
synopsisAndSentiment_prompt = """
As a professional psychologist, analyze the following transcript of a therapy session. Your objective is to generate two specific outputs:

1. A synopsis that succinctly captures the key concerns and topics discussed by the patient, providing insightful and reflective observations.
2. A detailed sentiment analysis of the patient's responses, identifying and elaborating on the specific emotions expressed.

Please format your output as a compact JSON object on a single line. The JSON should include two properties: 'Synopsis' and 'Sentiment', each containing the respective analyses. Avoid any extra spaces or line breaks within the JSON.

Example: "Synopsis":"Synopsis here", "Sentiment":"Sentiment here"

Transcript:
{Transcript}
"""


In [50]:
# Initialize summary list for storing JSON data
Synopsis = []

for index, row in merged_data.iterrows():
    prompt_multiplier = 1 
    for _ in range(prompt_multiplier):
        attempt_count = 0
        success = False
        while attempt_count < 10 and not success:
            attempt_count += 1
            # Format the prompt with the current row's conversation
            prompt = synopsisAndSentiment_prompt.format(Transcript=row['Conversation'])
            try:
                # Get synthetic data response
                response = get_response(prompt)
                print(f"Participant ID {row['ParticipantID']}")
                print(f"Attempt {attempt_count}: {response}")  

                # Try to parse the response as JSON
                json_data = json.loads(response)

                # Check if the required keys are present in the JSON data
                if 'Synopsis' in json_data and 'Sentiment' in json_data:
                    json_data['ParticipantID'] = row['ParticipantID']  
                    json_data['PHQ8_Score'] = row['PHQ_Score']
                    Synopsis.append(json_data)
                    success = True
                else:
                    print("Error: Required keys are missing from the JSON data. Retrying...")
                    clear_conversation()
                    continue  # Continue to retry since the JSON does not have the required keys

            except json.JSONDecodeError:
                # If JSON parsing fails, retry fetching the response
                print(f"Error: Response not loadable as JSON (Attempt {attempt_count}). Retrying...")
                clear_conversation()  
            except Exception as e:
                # Handle any other exceptions, including BadRequestError
                print(f"Error encountered: {e}. Skipping this prompt.")
                clear_conversation()  
                break

        clear_conversation()

# Convert the list of JSON objects to a DataFrame
synopsisAndSentiment_df = pd.DataFrame(Synopsis)
synopsisAndSentiment_df.to_csv('SynopsisAndSentimentWithEmotions.csv', index=False)

In [34]:
synopsisAndTranscript_df.shape

(519, 9)